In [106]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

In [107]:
my_table = soup.find('table',{'class':'wikitable sortable'})
my_table_body = my_table.findAll('tr')
postal = []
borough = []
neigh = []
for row in my_table_body[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if cols[1] != 'Not assigned':
        postal.append(cols[0])
        borough.append(cols[1])
        if cols[2] == 'Not assigned':
            neigh.append(cols[1])
        else:
            neigh.append(cols[2])
            


In [108]:
df = pd.DataFrame()
df['Postal Code'] = postal
df['Borough'] = borough
df['Neighborhood'] = neigh
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [109]:
df_loc = pd.read_csv("Geospatial_Coordinates.csv")
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [110]:
result = df.merge(df_loc,left_on='Postal Code',right_on='Postal Code')
result.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [111]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [118]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Mapping all the neighborhoods in Toronto

In [136]:
# create map of New York using latitude and longitude values
map_toronto= folium.Map(location=[latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [127]:
places = set()
for place in result['Borough']:
    if "Toronto" in place:
        places.add(place)

In [130]:
places = list(places)

In [131]:
places

['East Toronto', 'Downtown Toronto', 'West Toronto', 'Central Toronto']

In [138]:
map_toronto= folium.Map(location=[latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    
    if "Toronto" in borough:

        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=1,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
    
map_toronto


# Mapping all the Neighborhoods of Scarborough

In [139]:
CLIENT_ID = 'YNVDZFGUVDBLQZAQD3ZM5QYZTP0G5BRTSCY4DX4RBSGHCZCL'
CLIENT_SECRET = 'PUPENX4AXDYWJ2BFCRUQSTLV43AVNFXZP2B22JBRDGFPHE4M' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YNVDZFGUVDBLQZAQD3ZM5QYZTP0G5BRTSCY4DX4RBSGHCZCL
CLIENT_SECRET:PUPENX4AXDYWJ2BFCRUQSTLV43AVNFXZP2B22JBRDGFPHE4M


In [162]:
scarborough_data = result[result['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [163]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [165]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [175]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Getting All the nearby places of Scarborough

In [176]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude'])



Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [178]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000,0.0,0.000,0.000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.25,0.000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.333333,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.00,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,0.000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.000,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.25,0.000,0.000000,0.000000,0.25,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000,0.25,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.000000
3,Cedarbrae,0.000,0.0,0.125,0.000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.125,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000,0.000000,0.125000,0.125,0.00,0.000000,0.125,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000,0.125000,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.000,0.0,0.000,0.000,0.222222,0.000000,0.0,0.000000,0.000000,0.000000,0.222222,0.000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000,0.111111,0.000000,0.000,0.00,0.000000,0.000,0.000,0.000000,0.111111,0.000000,0.000000,0.00,0.00,0.000,0.111111,0.000,0.00,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.111111,0.000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.000,0.0,0.000,0.000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.076923,0.000000,0.00,0.076923,0.000,0.00,0.000,0.076923,0.076923,0.000,0.00,0.000000,0.000,0.000,0.000000,0.000000,0.076923,0.000000,0.00,0.00,0.000,0.000000,0.000,0.00,0.0,0.076923,0.000000,0.000000,0.153846,0.153846,0.000000,0.000,0.000000,0.076923,0.00,0.000000,0.000,0.076923,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.000,0.5,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.00,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.000,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000,0.00,0.000,0.166667,0.000000,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000,0.333333,0.000000,0.0000

In [179]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4          Accessories Store  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
          venue  freq
0          Park  0.33
1    Playground  0.33
2   Coffee Shop  0.33
3  Noodle House  0.00
4  Intersection  0.00


----Birch Cliff,Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4      Accessories Store  0.00


----Cedarbrae----
                 venue  freq
0     Hakka Restaurant  0.12
1  Fried Chicken Joint  0.12
2   Athletics & Sports  0.12
3      Thai Restaurant  0.12
4               Bakery  0.12


----Clairlea,Golden Mile,Oakridge----
                  venue  freq
0                Bakery  0.22
1              Bus Line  0.22
2          Intersection  0.11

In [182]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [183]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Coffee Shop,Park,Vietnamese Restaurant,Café,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff,Cliffside West",General Entertainment,Café,Skating Rink,College Stadium,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
3,Cedarbrae,Hakka Restaurant,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,College Stadium,General Entertainment
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Fast Food Restaurant,Soccer Field,Intersection,Park,Metro Station,Vietnamese Restaurant,Caribbean Restaurant,Electronics Store


# Clustering the places

In [184]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

In [185]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,4.0,Fast Food Restaurant,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Intersection,Rental Car Location,Medical Center,Breakfast Spot,Pizza Place,Mexican Restaurant,Electronics Store,Spa,Convenience Store,Department Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,College Stadium,General Entertainment


In [198]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
scarborough_merged.dropna(inplace = True)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters